In [1]:

from utils.dataloader import H5Dataset
from utils.EarlyStopping import EarlyStopping
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import classification_report
from tqdm import tqdm
from utils.dataloader import H5Dataset



In [2]:
num_classes = 6
k_folds = 10
num_epochs = 20



# Caminho para os arquivos h5
h5_files = ['h5_files/GunshotForensic_feature.h5',
            'h5_files/HEAR_Test_feature.h5',
            'h5_files/MIVIA_test_features.h5',
            'h5_files/MIVIA_train_features.h5',
            'h5_files/SESA_feature.h5',
            'h5_files/vsd_clipped_features.h5'
]

# Carregando os dataloaders
dataset = H5Dataset(h5_files, exclude_classes=[6], n_splits=k_folds)
dataloaders = dataset.get_k_fold_data_loaders(batch_size=32,
                                               num_workers=4)


In [3]:
from torchvision.models import ResNet101_Weights


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

criterion = nn.CrossEntropyLoss()

# Configurando o TensorBoard
writer = SummaryWriter()





cuda


In [4]:
# Loop de treinamento
for fold, (train_loader, val_loader) in enumerate(dataloaders):
    print(f'Starting fold {fold + 1}/{len(dataloaders)}')
    
    # Ajustando a ResNet101
    model = models.resnet101(weights=ResNet101_Weights.IMAGENET1K_V1)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_classes)  # ajustar a última camada para 6 classes
    model.conv1 = nn.Conv2d(1,
                            64,
                            kernel_size=(7, 7),
                            stride=(2, 2),
                            padding=(3, 3),
                            bias=False)  # ajustar a primeira camada para 1 canal
    model = model.to(device)
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    # initialize the early_stopping object
    early_stopping = EarlyStopping(patience=7, verbose=True, fold = fold)
    
    for epoch in range(num_epochs):  # número de épocas
        print(f'Starting epoch {epoch + 1}/{num_epochs}')

        # Treinamento
        model.train()
        running_loss_train = 0.0
        running_corrects_train = 0
        for i, (inputs, labels) in tqdm(enumerate(train_loader), total=len(train_loader)):
            inputs = inputs.unsqueeze(1).to(device)  # Adiciona um canal extra
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            running_corrects_train += torch.sum(preds == labels.data)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss_train += loss.item() * inputs.size(0)

        epoch_loss_train = running_loss_train / len(train_loader.dataset)
        epoch_acc_train = running_corrects_train.double() / len(train_loader.dataset)

        # Validação
        model.eval()
        running_loss_val = 0.0
        running_corrects_val = 0
        all_labels = []
        all_preds = []
        with torch.no_grad():
            for inputs, labels in tqdm(val_loader):
                inputs = inputs.unsqueeze(1).to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                running_corrects_val += torch.sum(preds == labels.data)
                loss = criterion(outputs, labels)

                running_loss_val += loss.item() * inputs.size(0)

                all_labels.extend(labels.data.cpu().numpy())
                all_preds.extend(preds.cpu().numpy())

        epoch_loss_val = running_loss_val / len(val_loader.dataset)
        epoch_acc_val = running_corrects_val.double() / len(val_loader.dataset)

        report = classification_report(all_labels, all_preds, output_dict=True)

        print(f'Train Loss: {epoch_loss_train:.4f} Train Acc: {epoch_acc_train:.4f}')
        print(f'Validation Loss: {epoch_loss_val:.4f} Validation Acc: {epoch_acc_val:.4f}')

        f1_scores = {}
        precisions = {}
        recalls = {}
        for class_index in range(num_classes):
            f1_scores[f'class_{class_index}'] = report[str(class_index)]['f1-score']
            precisions[f'class_{class_index}'] = report[str(class_index)]['precision']
            recalls[f'class_{class_index}'] = report[str(class_index)]['recall']

        writer.add_scalars(f'F1 scores_{fold}', f1_scores, epoch)
        writer.add_scalars(f'Precisions_{fold}', precisions, epoch)
        writer.add_scalars(f'Recalls_{fold}', recalls, epoch)

        writer.add_scalar(f'training loss_{fold}', epoch_loss_train, epoch)
        writer.add_scalar(f'training accuracy_{fold}', epoch_acc_train, epoch)
        writer.add_scalar(f'validation loss_{fold}', epoch_loss_val, epoch)
        writer.add_scalar(f'validation accuracy_{fold}', epoch_acc_val, epoch)
        
        early_stopping(epoch_loss_val, model)
        
        if early_stopping.early_stop:
            print("Early stopping")
            break

    print(f'Finished fold {fold + 1}/{len(dataloaders)}')
    
# Salvando o modelo treinado
torch.save(model.state_dict(), f'models/model_resnet101_{k_folds}_Folds.pth')

# Fechando o TensorBoard writer
writer.close()

print('Finished Training')


Starting fold 1/10
Starting epoch 1/20


100%|██████████| 113/113 [01:02<00:00,  1.81it/s]


Train Loss: 0.5951 Train Acc: 0.7449
Validation Loss: 1.3221 Validation Acc: 0.6310
Validation loss decreased (inf --> 1.322066).  Saving model ...
Starting epoch 2/20


100%|██████████| 113/113 [01:01<00:00,  1.84it/s]


Train Loss: 0.5044 Train Acc: 0.7839
Validation Loss: 1.6615 Validation Acc: 0.4969
EarlyStopping counter: 1 out of 7
Starting epoch 3/20


...

Train Loss: 0.3889 Train Acc: 0.8270
Validation Loss: 0.9684 Validation Acc: 0.7580
EarlyStopping counter: 5 out of 7
Starting epoch 7/20


100%|██████████| 113/113 [01:01<00:00,  1.84it/s]


Train Loss: 0.3707 Train Acc: 0.8382
Validation Loss: 0.9742 Validation Acc: 0.7510
EarlyStopping counter: 6 out of 7
Starting epoch 8/20


100%|██████████| 113/113 [01:01<00:00,  1.83it/s]


Train Loss: 0.3554 Train Acc: 0.8448
Validation Loss: 1.0228 Validation Acc: 0.7319
EarlyStopping counter: 7 out of 7
Early stopping
Finished fold 10/10
Finished Training


In [5]:
import numpy as np 
import os
# Dicionário para armazenar os relatórios de classificação para cada fold
classification_reports = []

# Caminho para a pasta com os modelos
model_folder = 'models/checkpoint'

# Loop através de cada fold
for fold in range(k_folds):
    # Carregando o modelo
    model_path = os.path.join(model_folder, f'checkpoint_{fold}.pth')
    model.load_state_dict(torch.load(model_path))

    # Pegando o dataloader de teste para este fold
    _, test_loader = dataloaders[fold]

    # Avaliando o modelo neste fold
    model.eval()
    all_labels = []
    all_preds = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.unsqueeze(1).to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    # Gerando o relatório de classificação para este fold
    report = classification_report(all_labels, all_preds, output_dict=True)
    classification_reports.append(report)

# Calculando a média das métricas para cada classe em todos os folds
avg_metrics = {class_: {'precision': 0, 'recall': 0, 'f1-score': 0, 'support': 0} for class_ in range(num_classes)}
for report in classification_reports:
    for class_ in range(num_classes):
        avg_metrics[class_]['precision'] += report[str(class_)]['precision']
        avg_metrics[class_]['recall'] += report[str(class_)]['recall']
        avg_metrics[class_]['f1-score'] += report[str(class_)]['f1-score']
        avg_metrics[class_]['support'] += report[str(class_)]['support']

for class_ in range(num_classes):
    avg_metrics[class_]['precision'] /= k_folds
    avg_metrics[class_]['recall'] /= k_folds
    avg_metrics[class_]['f1-score'] /= k_folds
    avg_metrics[class_]['support'] = np.round(avg_metrics[class_]['support'] / k_folds).astype

In [6]:
annotation_dict = {0: 'Nada',
                   1: 'Grito',
                   2: 'Violência Física',
                   3: 'Tiro',
                   4: 'Explosão',
                   5: 'Grito de Esforço'}

for class_ in range(num_classes):
    print(f"Class: {annotation_dict[class_]}")
    print('Precision\tRecall\tF1-Score\tSupport')
    print('{:.4f}\t\t{:.4f}\t{:.4f}\t\t{}'.format(avg_metrics[class_]['precision'], 
                                                  avg_metrics[class_]['recall'], 
                                                  avg_metrics[class_]['f1-score'], 
                                                  avg_metrics[class_]['support'](int)))
    print('===============================================')

Class: Nada
Precision	Recall	F1-Score	Support
0.8514		0.8733	0.8605		2084
Class: Grito
Precision	Recall	F1-Score	Support
0.3443		0.2287	0.2321		324
Class: Violência Física
Precision	Recall	F1-Score	Support
0.8769		0.8298	0.8406		500
Class: Tiro
Precision	Recall	F1-Score	Support
0.9026		0.7844	0.7942		315
Class: Explosão
Precision	Recall	F1-Score	Support
0.6562		0.3295	0.3825		65
Class: Grito de Esforço
Precision	Recall	F1-Score	Support
0.5792		0.5530	0.4863		316
